In [ ]:
import sys
sys.path.append("../../PreProcessing")
sys.path.append("../../features")

from PreProcessing import PreProcessing
from ExtractFeatures import extract_features
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import skimage as ski
from sklearn.model_selection import train_test_split
import tqdm
import pandas as pd
import os
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score



Model Training Pipeline

In [ ]:
# 1. Preprocess the images
# 2. Extract features
# 3. Train the model
# 4. Test the model

def main():

    folder_name = ["IBM Plex Sans Arabic" ,"Lemonada" , "Marhey" , "Scheherazade New" ]

    # for j in range(0 , 4 ) : 
    #     for i in tqdm(range(0, 1000)): 
    #         img = ski.io.imread("../../data/raw/fonts-dataset/"+folder_name[j]+"/"+str(i)+".jpeg")
            
    #         if len(np.unique(img)) == 1 : 
    #             continue # all image contain the same color 

    #         preprocessed_image = PreProcessing(img)
    #         ski.io.imsave("../../data/processed/fonts-dataset/"+folder_name[j]+"/"+str(i)+".jpeg", preprocessed_image)

    # 2. Extract features
    # Extracting features
    columns = ["area_ratio"]
    for j in range(1 , 17):
        columns.append("gabor_mean_" + str(j))
        columns.append("gabor_std_" + str(j))
        columns.append("gabor_energy_" + str(j))
    columns.append("font_type")
    features = pd.DataFrame(columns=columns)
    for j in range(0 , 4):
        for i in tqdm.tqdm(range(0, 1000)):
            pathname = "../../data/processed/fonts-dataset/"+folder_name[j]+"/"+str(i)+".jpeg"
            if os.path.exists(pathname):
                img = ski.io.imread(pathname)
                extractFeatures = extract_features(img)
                features = pd.concat([features , pd.DataFrame([np.concatenate([extractFeatures , [j]])] , columns = columns)])


    features.to_csv('./features_1000.csv')

    # 3. Train the model
    dataset = pd.read_csv('./features_1000.csv')

    # column split
    x_data = dataset.drop(columns=['Unnamed: 0'])     
    x_data = x_data.iloc[:, :-1]                                   
    y_data = dataset['font_type']                              

    #train-test split
    x_data, y_data = x_data.to_numpy(), y_data.to_numpy()
    x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=75, stratify=y_data)

    # Normalize the data
    scaler = StandardScaler().fit(x_train)
    x_train = scaler.transform(x_train) 

    svm_model = svm.SVC(kernel='poly', C=10, gamma='scale', degree=3, coef0=1)
    svm_model.fit(x_train, y_train)

    # 4. Test the model
    y_pred = svm_model.predict(x_val)
    accuracy = accuracy_score(y_val, y_pred)
    print("svm Accuracy:", accuracy)



main()